In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
application_df = pd.read_csv('Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME' and 'SPECIAL_CONSIDERATIONS'. --ADD SPECIAL_CONSIDERATIONS
application_df = application_df.drop(columns=["EIN","NAME","SPECIAL_CONSIDERATIONS"], axis=1)
print(application_df.shape)
application_df.head(10)

(34299, 9)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0


In [3]:
# Only keep rows with STATUS of 1--ADDED THIS 

application_df = application_df[application_df['STATUS'] == 1]
print(application_df.shape)
application_df.head(10)

(34294, 9)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0


In [4]:
## REMOVED STATUS COLUMN AS IT IS NO LONGER NECESSARY
application_df = application_df.drop(columns=["STATUS"], axis=1)
print(application_df.shape)
application_df.head(10)

(34294, 8)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
5,T3,Independent,C1200,Preservation,Trust,0,5000,1
6,T3,Independent,C1000,Preservation,Trust,100000-499999,31452,1
7,T3,Independent,C2000,Preservation,Trust,10M-50M,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1-9999,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df.APPLICATION_TYPE.value_counts()
app_type_counts

T3     27032
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27032
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
print(classification_counts)

C1000    17323
C2000     6073
C1200     4837
C3000     1918
C2100     1882
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [9]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17323
C2000     6073
C1200     4837
Other     2261
C3000     1918
C2100     1882
Name: CLASSIFICATION, dtype: int64

In [10]:
# Look at ASK_AMT value counts for binning --ADDED THIS
ask_counts = application_df.ASK_AMT.value_counts()
print(ask_counts)


#ask_amt_counts = application_df.ASK_AMT.value_counts(bins=8)
#ask_amt_counts = application_df["ASK_AMT"].min()
#ask_amt_counts

5000        25394
63981           3
15583           3
6725            3
10478           3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [11]:
# Determine which values to replace if counts are less than ..? --BINNING ASK AMOUNTS -ADDED THIS
replace_ask = list(ask_counts[ask_counts <= 100].index)

for ask in replace_ask:
    if ask <=10000:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,"5000-10000")
    elif ask <=50000:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,"10000-50000")
    elif ask <=100000:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,"50000-100000")
    elif ask <=500000:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,"100000-500000")
    elif ask <=1000000:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,"500000-1000000")
    elif ask <=10000000:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,"1000000-10000000")
    elif ask <=100000000:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,"10000000-100000000")
    else:
         application_df.ASK_AMT = application_df.ASK_AMT.replace(ask,">100000000")        

# Check to make sure binning was successful
application_df.ASK_AMT.value_counts()



5000                  25394
10000-50000            2398
100000-500000          2304
50000-100000           1423
1000000-10000000       1165
500000-1000000          650
5000-10000              548
10000000-100000000      311
>100000000              101
Name: ASK_AMT, dtype: int64

In [12]:
application_df['ASK_AMT']=application_df['ASK_AMT'].astype(str)

In [13]:
application_df.dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
INCOME_AMT          object
ASK_AMT             object
IS_SUCCESSFUL        int64
dtype: object

In [14]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'ASK_AMT']

In [15]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\aaron\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_5M-10M,ASK_AMT_10000-50000,ASK_AMT_100000-500000,ASK_AMT_1000000-10000000,ASK_AMT_10000000-100000000,ASK_AMT_5000,ASK_AMT_5000-10000,ASK_AMT_50000-100000,ASK_AMT_500000-1000000,ASK_AMT_>100000000
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True)
application_df = application_df.drop(columns=application_cat, axis=1)
application_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_5M-10M,ASK_AMT_10000-50000,ASK_AMT_100000-500000,ASK_AMT_1000000-10000000,ASK_AMT_10000000-100000000,ASK_AMT_5000,ASK_AMT_5000-10000,ASK_AMT_50000-100000,ASK_AMT_500000-1000000,ASK_AMT_>100000000
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\aaron\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3920      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,381
Trainable params: 6,381
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=10)

In [22]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 660us/step - loss: 0.6841 - accuracy: 0.5573
Epoch 2/100
804/804 [==============================] - 1s 685us/step - loss: 0.6806 - accuracy: 0.5703
Epoch 3/100
804/804 [==============================] - 1s 686us/step - loss: 0.6792 - accuracy: 0.5719
Epoch 4/100
804/804 [==============================] - 1s 654us/step - loss: 0.6785 - accuracy: 0.5733
Epoch 5/100
804/804 [==============================] - 1s 644us/step - loss: 0.6777 - accuracy: 0.5758
Epoch 6/100
804/804 [==============================] - 1s 657us/step - loss: 0.6770 - accuracy: 0.5770
Epoch 7/100
804/804 [==============================] - 1s 652us/step - loss: 0.6762 - accuracy: 0.5765
Epoch 8/100
804/804 [==============================] - 1s 644us/step - loss: 0.6756 - accuracy: 0.5770
Epoch 9/100
804/804 [==============================] - 1s 640us/step - loss: 0.6748 - accuracy: 0.5797
Epoch 10/100
787/804 [============================>.] - ETA: 0s - loss: 0

Epoch 71/100
804/804 [==============================] - 1s 702us/step - loss: 0.6552 - accuracy: 0.5943
Epoch 72/100
804/804 [==============================] - 1s 666us/step - loss: 0.6552 - accuracy: 0.5958
Epoch 73/100
804/804 [==============================] - 1s 659us/step - loss: 0.6551 - accuracy: 0.5957
Epoch 74/100
804/804 [==============================] - 1s 649us/step - loss: 0.6556 - accuracy: 0.5965
Epoch 75/100
804/804 [==============================] - 1s 654us/step - loss: 0.6547 - accuracy: 0.5965
Epoch 76/100
804/804 [==============================] - 1s 652us/step - loss: 0.6550 - accuracy: 0.5955
Epoch 77/100
804/804 [==============================] - 1s 678us/step - loss: 0.6548 - accuracy: 0.5933
Epoch 78/100
804/804 [==============================] - 1s 648us/step - loss: 0.6550 - accuracy: 0.5941
Epoch 79/100
804/804 [==============================] - 1s 664us/step - loss: 0.6547 - accuracy: 0.5960
Epoch 80/100
753/804 [===========================>..] - ETA: 0s 

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 5.1286 - accuracy: 0.5282 - 223ms/epoch - 832us/step
Loss: 5.1285624504089355, Accuracy: 0.5281698107719421


In [24]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_firstattempt.h5")

In [25]:
# redefining model with more hidden nodes

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3920      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 10)                310       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6,671
Trainable params: 6,671
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 741us/step - loss: 0.6842 - accuracy: 0.5604
Epoch 2/100
804/804 [==============================] - 1s 738us/step - loss: 0.6802 - accuracy: 0.5707
Epoch 3/100
804/804 [==============================] - 1s 732us/step - loss: 0.6793 - accuracy: 0.5714
Epoch 4/100
804/804 [==============================] - 1s 715us/step - loss: 0.6787 - accuracy: 0.5766
Epoch 5/100
804/804 [==============================] - 1s 728us/step - loss: 0.6779 - accuracy: 0.5743
Epoch 6/100
804/804 [==============================] - 1s 695us/step - loss: 0.6770 - accuracy: 0.5785
Epoch 7/100
804/804 [==============================] - 1s 701us/step - loss: 0.6765 - accuracy: 0.5763
Epoch 8/100
804/804 [==============================] - 1s 705us/step - loss: 0.6763 - accuracy: 0.5771
Epoch 9/100
804/804 [==============================] - 1s 726us/step - loss: 0.6751 - accuracy: 0.5801
Epoch 10/100
770/804 [===========================>..] - ETA: 0s - loss: 0

Epoch 71/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6542 - accuracy: 0.5940
Epoch 72/100
804/804 [==============================] - 1s 920us/step - loss: 0.6538 - accuracy: 0.5962
Epoch 73/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6538 - accuracy: 0.5946
Epoch 74/100
804/804 [==============================] - 1s 946us/step - loss: 0.6539 - accuracy: 0.5953
Epoch 75/100
804/804 [==============================] - 1s 971us/step - loss: 0.6530 - accuracy: 0.5961
Epoch 76/100
804/804 [==============================] - 1s 942us/step - loss: 0.6538 - accuracy: 0.5956
Epoch 77/100
804/804 [==============================] - 1s 964us/step - loss: 0.6539 - accuracy: 0.5952
Epoch 78/100
804/804 [==============================] - 1s 949us/step - loss: 0.6534 - accuracy: 0.5955
Epoch 79/100
804/804 [==============================] - 1s 925us/step - loss: 0.6532 - accuracy: 0.5966
Epoch 80/100
803/804 [============================>.] - ETA: 0s - lo

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 9.3352 - accuracy: 0.5255 - 249ms/epoch - 929us/step
Loss: 9.335193634033203, Accuracy: 0.5254870057106018


In [29]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_secondattempt.h5")

In [30]:
# redefining model with more hidden node layers and neurons per layer

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 130
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20
hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 130)               6370      
                                                                 
 dense_8 (Dense)             (None, 40)                5240      
                                                                 
 dense_9 (Dense)             (None, 20)                820       
                                                                 
 dense_10 (Dense)            (None, 10)                210       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 12,651
Trainable params: 12,651
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 938us/step - loss: 0.6842 - accuracy: 0.5610
Epoch 2/100
804/804 [==============================] - 1s 916us/step - loss: 0.6811 - accuracy: 0.5685
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6799 - accuracy: 0.5700
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6791 - accuracy: 0.5732
Epoch 5/100
804/804 [==============================] - 1s 986us/step - loss: 0.6783 - accuracy: 0.5744
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6777 - accuracy: 0.5745
Epoch 7/100
804/804 [==============================] - 1s 993us/step - loss: 0.6771 - accuracy: 0.5760
Epoch 8/100
804/804 [==============================] - 1s 970us/step - loss: 0.6763 - accuracy: 0.5757
Epoch 9/100
804/804 [==============================] - 1s 990us/step - loss: 0.6754 - accuracy: 0.5792
Epoch 10/100
794/804 [============================>.] - ETA: 0s - loss: 0.6747 

804/804 [==============================] - 1s 1ms/step - loss: 0.6509 - accuracy: 0.5975
Epoch 72/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6508 - accuracy: 0.5980
Epoch 73/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6516 - accuracy: 0.5970
Epoch 74/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6508 - accuracy: 0.5977
Epoch 75/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6507 - accuracy: 0.5993
Epoch 76/100
804/804 [==============================] - 1s 972us/step - loss: 0.6503 - accuracy: 0.5987
Epoch 77/100
804/804 [==============================] - 1s 957us/step - loss: 0.6503 - accuracy: 0.5981
Epoch 78/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6504 - accuracy: 0.5963
Epoch 79/100
804/804 [==============================] - 1s 990us/step - loss: 0.6509 - accuracy: 0.5979
Epoch 80/100
753/804 [===========================>..] - ETA: 0s - loss: 0.6491 - accuracy

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 20.2078 - accuracy: 0.5157 - 362ms/epoch - 1ms/step
Loss: 20.207813262939453, Accuracy: 0.5156887769699097


In [34]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_thirdattempt.h5")

In [35]:
# redefining model with different activation functions

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 130
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20
hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 130)               6370      
                                                                 
 dense_13 (Dense)            (None, 40)                5240      
                                                                 
 dense_14 (Dense)            (None, 20)                820       
                                                                 
 dense_15 (Dense)            (None, 10)                210       
                                                                 
 dense_16 (Dense)            (None, 1)                 11        
                                                                 
Total params: 12,651
Trainable params: 12,651
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 798us/step - loss: 0.6853 - accuracy: 0.5605
Epoch 2/100
804/804 [==============================] - 1s 902us/step - loss: 0.6820 - accuracy: 0.5646
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6805 - accuracy: 0.5671
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6799 - accuracy: 0.5697
Epoch 5/100
804/804 [==============================] - 1s 962us/step - loss: 0.6791 - accuracy: 0.5736
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6791 - accuracy: 0.5694
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6784 - accuracy: 0.5735
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6784 - accuracy: 0.5751
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6776 - accuracy: 0.5737
Epoch 10/100
760/804 [===========================>..] - ETA: 0s - loss: 0.6779 - accu

804/804 [==============================] - 1s 1ms/step - loss: 0.6608 - accuracy: 0.5929
Epoch 72/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6605 - accuracy: 0.5918
Epoch 73/100
804/804 [==============================] - 1s 992us/step - loss: 0.6603 - accuracy: 0.5952
Epoch 74/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6604 - accuracy: 0.5935
Epoch 75/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6599 - accuracy: 0.5958
Epoch 76/100
804/804 [==============================] - 1s 953us/step - loss: 0.6597 - accuracy: 0.5934
Epoch 77/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6596 - accuracy: 0.5925
Epoch 78/100
804/804 [==============================] - 1s 994us/step - loss: 0.6598 - accuracy: 0.5929
Epoch 79/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6595 - accuracy: 0.5945
Epoch 80/100
789/804 [============================>.] - ETA: 0s - loss: 0.6594 - accuracy

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8390 - accuracy: 0.5455 - 287ms/epoch - 1ms/step
Loss: 0.8390052914619446, Accuracy: 0.5455499887466431


In [39]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_fourthattempt.h5")

In [40]:
# redefining model with different activation functions and only 2 hidden layers

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 100)               4900      
                                                                 
 dense_18 (Dense)            (None, 30)                3030      
                                                                 
 dense_19 (Dense)            (None, 1)                 31        
                                                                 
Total params: 7,961
Trainable params: 7,961
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [42]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 725us/step - loss: 0.6874 - accuracy: 0.5542
Epoch 2/100
804/804 [==============================] - 1s 890us/step - loss: 0.6831 - accuracy: 0.5621
Epoch 3/100
804/804 [==============================] - 1s 932us/step - loss: 0.6822 - accuracy: 0.5672
Epoch 4/100
804/804 [==============================] - 1s 953us/step - loss: 0.6814 - accuracy: 0.5684
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6808 - accuracy: 0.5714
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6799 - accuracy: 0.5704
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6795 - accuracy: 0.5711
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6791 - accuracy: 0.5748
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6788 - accuracy: 0.5731
Epoch 10/100
763/804 [===========================>..] - ETA: 0s - loss: 0.6784 - ac

Epoch 71/100
804/804 [==============================] - 1s 772us/step - loss: 0.6701 - accuracy: 0.5866
Epoch 72/100
804/804 [==============================] - 1s 880us/step - loss: 0.6698 - accuracy: 0.5857
Epoch 73/100
804/804 [==============================] - 1s 836us/step - loss: 0.6697 - accuracy: 0.5864
Epoch 74/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6693 - accuracy: 0.5857
Epoch 75/100
804/804 [==============================] - 1s 679us/step - loss: 0.6693 - accuracy: 0.5884
Epoch 76/100
804/804 [==============================] - 1s 839us/step - loss: 0.6693 - accuracy: 0.5845
Epoch 77/100
804/804 [==============================] - 1s 756us/step - loss: 0.6688 - accuracy: 0.5862
Epoch 78/100
804/804 [==============================] - 1s 842us/step - loss: 0.6691 - accuracy: 0.5879
Epoch 79/100
804/804 [==============================] - 1s 868us/step - loss: 0.6687 - accuracy: 0.5894
Epoch 80/100
753/804 [===========================>..] - ETA: 0s - 

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7202 - accuracy: 0.5457 - 305ms/epoch - 1ms/step
Loss: 0.7202345132827759, Accuracy: 0.5456666350364685


In [44]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_fifthattempt.h5")